In [5]:
import os 
import tokenizer
import librosa
import pandas as pd
import jiwer
from huggingface_hub import notebook_login
!pip install datasets==1.18.3
!pip install transformers==4.17.0
notebook_login()
import sys
List = sys.path
print(list)

<class 'list'>


In [6]:

df_train = pd.read_csv('DataSets/TIMIT/train_data.csv')
df_train = df_train[:8400]
df_train





,index,test_or_train,dialect_region,speaker_id,filename,path_from_data_dir,path_from_data_dir_windows,is_converted_audio,is_audio,is_word_file,is_phonetic_file,is_sentence_file
0,1.0,TRAIN,DR4,MMDM0,SI681.WAV.wav,TRAIN/DR4/MMDM0/SI681.WAV.wav,TRAIN\\DR4\\MMDM0\\SI681.WAV.wav,True,True,False,False,False
1,2.0,TRAIN,DR4,MMDM0,SI1311.PHN,TRAIN/DR4/MMDM0/SI1311.PHN,TRAIN\\DR4\\MMDM0\\SI1311.PHN,False,False,False,True,False
2,3.0,TRAIN,DR4,MMDM0,SI1311.WRD,TRAIN/DR4/MMDM0/SI1311.WRD,TRAIN\\DR4\\MMDM0\\SI1311.WRD,False,False,True,False,False
3,4.0,TRAIN,DR4,MMDM0,SX321.PHN,TRAIN/DR4/MMDM0/SX321.PHN,TRAIN\\DR4\\MMDM0\\SX321.PHN,False,False,False,True,False
4,5.0,TRAIN,DR4,MMDM0,SX321.WRD,TRAIN/DR4/MMDM0/SX321.WRD,TRAIN\\DR4\\MMDM0\\SX321.WRD,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8395,8396.0,TRAIN,DR2,MMAA0,SA2.WAV.wav,TRAIN/DR2/MMAA0/SA2.WAV.wav,TRAIN\\DR2\\MMAA0\\SA2.WAV.wav,True,True,False,False,False
8396,8397.0,TRAIN,DR2,MMAA0,SX125.WAV.wav,TRAIN/DR2/MMAA0/SX125.WAV.wav,TRAIN\\DR2\\MMAA0\\SX125.WAV.wav,True,True,False,False,False
8397,8398.0,TRAIN,DR2,MMAA0,SI845.WAV,TRAIN/DR2/MMAA0/SI845.WAV,TRAIN\\DR2\\MMAA0\\SI845.WAV,False,True,False,False,False
8398,8399.0,TRAIN,DR2,MMAA0,SX35.TXT,TRAIN/DR2/MMAA0/SX35.TXT,TRAIN\\DR2\\MMAA0\\SX35.TXT,False,False,False,False,True


In [7]:
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")


Reusing dataset timit_asr (/Users/mac/.cache/huggingface/datasets/timit_asr/clean/2.0.1/b11b576ddcccbcefa7c9f0c4e6c2a43756f3033adffe0fb686aa61043d0450ad)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])
timit


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 1680
    })
})

In [9]:
from datasets import ClassLabel
import random

from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(timit["train"].remove_columns(["file", "audio"]))

,text
0,Now he'll choke for sure.
1,Originals are not necessarily good and adaptations are not necessarily bad.
2,And the law virtually ignored the situation.
3,She had your dark suit in greasy wash water all year.
4,If we left one we'd have to wipe it for fingerprints.
5,She had your dark suit in greasy wash water all year.
6,It's not easy to create illuminating examples.
7,Can't seem to locate landmarks in this snow.
8,Index words and electronic switches may be reserved in the following ways.
9,Regular attendance is seldom required.


In [10]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

timit = timit.map(remove_special_characters)


Parameter 'function'=<function remove_special_characters at 0x127b5c0e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [11]:
show_random_elements(timit["train"].remove_columns(["file", "audio"]))

,text
0,in the long run it pays to buy quality clothing
1,the cigarettes in the clay ashtray overflowed onto the oak table
2,don't lose the chuck key
3,his comrades were all dead
4,his talk turns to what he calls the mess or sometimes this buzzing confusion
5,the sheriff's swivel chair tilted back
6,with this no loyal citizen can quarrel
7,it takes a great deal of sophisticated thought to get the impact of this fact
8,when peeling an orange it is hard not to spray juice
9,a huge power outage rarely occurs


In [12]:
def extract_all_chars(batch):
    
  all_text = " ".join(batch["text"])
  b = set(all_text)
    
  vocab = list(b)
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict


{'d': 0,
 't': 1,
 'a': 2,
 'j': 3,
 'u': 4,
 'c': 5,
 's': 6,
 'k': 7,
 'g': 8,
 'm': 9,
 'o': 10,
 "'": 11,
 'v': 12,
 'b': 13,
 'n': 14,
 'f': 15,
 'l': 16,
 'y': 17,
 'q': 18,
 'r': 19,
 'p': 20,
 'w': 21,
 'h': 22,
 'e': 23,
 ' ': 24,
 'x': 25,
 'i': 26,
 'z': 27}

In [14]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [15]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))


30


In [16]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [17]:
repo_name = "wav2vec2-base-timit-demo-colab"

In [18]:
tokenizer.push_to_hub(repo_name)

AttributeError: module 'tokenizer' has no attribute 'push_to_hub'

In [19]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [20]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)


In [21]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [22]:
timit['train']['audio']


[{'path': '/Users/mac/.cache/huggingface/datasets/downloads/extracted/86096937f792a33c316ac9ea43f48d3dc40aac8eb909880575f0f2eb2fc29ded/data/TRAIN/DR4/MMDM0/SI681.WAV',
  'array': array([-2.1362305e-04,  6.1035156e-05,  3.0517578e-05, ...,
         -3.0517578e-05, -9.1552734e-05, -6.1035156e-05], dtype=float32),
  'sampling_rate': 16000},
 {'path': '/Users/mac/.cache/huggingface/datasets/downloads/extracted/86096937f792a33c316ac9ea43f48d3dc40aac8eb909880575f0f2eb2fc29ded/data/TRAIN/DR4/MMDM0/SA2.WAV',
  'array': array([-3.6621094e-04,  9.1552734e-05, -3.0517578e-05, ...,
          3.0517578e-05, -6.1035156e-05,  0.0000000e+00], dtype=float32),
  'sampling_rate': 16000},
 {'path': '/Users/mac/.cache/huggingface/datasets/downloads/extracted/86096937f792a33c316ac9ea43f48d3dc40aac8eb909880575f0f2eb2fc29ded/data/TRAIN/DR4/MMDM0/SX411.WAV',
  'array': array([ 2.7465820e-04, -9.1552734e-05,  3.0517578e-05, ...,
          0.0000000e+00,  3.0517578e-05, -3.0517578e-05], dtype=float32),
  'sampli

In [23]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)


there is no dispute involving fieldwork


In [24]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])



Target text: she had your dark suit in greasy wash water all year
Input array shape: (57140,)
Sampling rate: 16000


In [41]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch


In [43]:
import numpy as np
np.object = object 
timit = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=4)


KeyError: 'audio'

In [46]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [47]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [48]:
wer_metric = load_metric("wer")


In [49]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [50]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)


/Users/mac/anaconda3/lib/python3.11/site-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/Users/mac/anaconda3/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_q.weight', 'project_hid.weight', 'project_q.bias']
- This IS expect

In [51]:
model.freeze_feature_extractor()


/Users/mac/anaconda3/lib/python3.11/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1709: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [52]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)


In [34]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit_prepared["train"],
    eval_dataset=timit_prepared["test"],
    tokenizer=processor.feature_extractor,
)


NameError: name 'data_collator' is not defined